In [464]:

from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
 
# import twitter_credentials
import numpy as np
import pandas as pd
consumer_key = "cFf7im7BH68xO9qh3zEsv3nFz"
consumer_secret = "2QyodEVK63XYE5D9RFPAo0I53rhBOsNocQGpGB8rapmEqxDnJi"
access_token = "1265319352795975680-EJBIU55ZHZnjn8svR420cqVuU9evRL"
access_token_secret = "i8h2utF6b2l2Uh7Vpg6c2mnPOO1CuthGdbyWtHSWdIURP"



# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        return auth

# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)


class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])
        df['Date Created'] = np.array([tweet.created_at for tweet in tweets])
        df['id'] = np.array([tweet.id for tweet in tweets])
        df['likes'] = np.array([tweet.favorite_count for tweet in tweets])
        df['retweets'] = np.array([tweet.retweet_count for tweet in tweets])

        return df

 
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="narendramodi", count=200)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df.to_csv('{}-tweets.csv')


In [465]:
df

,Tweets,Date Created,id,likes,retweets
0,RT @AmitShah: कांग्रेस ने 60साल तक बस गरीबी हट...,2020-06-07 16:07:08,1269662183371177985,0,7844
1,"RT @AmitShah: भाजपा लोकतंत्र, जनसंपर्क और जनसं...",2020-06-07 16:07:01,1269662156687097857,0,3102
2,Shri Ved Marwah Ji will be remembered for his ...,2020-06-06 06:01:16,1269147325571588097,60232,6434
3,"Rwanda is, and will remain, an important pilla...",2020-06-05 14:28:30,1268912585790951424,21940,2493
4,"Thank you, my friend President @PaulKagame. \n...",2020-06-05 14:28:29,1268912582267678720,36511,4608
...,...,...,...,...,...
195,Please thank Swetha for this creative and affe...,2020-04-27 17:59:11,1254832480189702144,76377,8435
196,Good gesture by the @DelhiPolice.\n\nOur Gurud...,2020-04-27 15:12:09,1254790442882760704,67084,10435
197,Today was the 4th interaction with CMs. We con...,2020-04-27 12:00:17,1254742160131321857,62883,7512
198,Shared my thoughts about the rich and noble id...,2020-04-26 16:06:30,1254441735393964034,42508,5387


In [456]:
df.drop(columns=["id"],axis=1,inplace=True)
df.drop(df[df['likes']<=25000].index,inplace=True)
df.drop(df[df['retweets']>=5500].index,inplace=True)
df.drop(df[~df['Tweets'].str.contains("COVID-19")].index).count()
df.index = [x for x in range(df["Tweets"].count())]

In [457]:
n_tweet = []

In [458]:
from datetime import datetime, timedelta
n_tweet = []
def filtered_date(x):
    time = datetime.now()
    if (time - x) > timedelta(1): 
        if (time-x).days ==1:
            n_tweet.append("new tweet")
        else:
            n_tweet.append("old tweet")
# df['New Tweet'] = n_tweet

In [459]:
len(n_tweet)

0

In [460]:
s = df['Date Created']
s.index = [x for x in range(s.count())]
s.apply(filtered_date)

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25    None
26    None
27    None
28    None
29    None
30    None
31    None
32    None
33    None
34    None
35    None
36    None
37    None
38    None
39    None
Name: Date Created, dtype: object

In [ ]:
df1 = pd.DataFrame({'Tweet Update':n_tweet})
df = pd.concat([df,df1],axis=1)

In [462]:
df
df.to_csv('filtered.csv')